# Resumen de la informacion relevante de cada estacion

El objetivo de este codigo es obtener una tabla resumen con la relacion n-n entre estaciones de calidad del aire y meteorologicas que ademas muestre cuanta informacion relevante aporta cada estacion. A partir de dicha tabla se espera decidir que estaciones aportan mas datos al estudio.

PUESTO QUE SE TRATA DE UNA GRAN CANTIDAD DE DATOS Y REALIZAR TODA LA EJECUCION LLEVARIA DEMASIADO TIEMPO, EN ESTE NOTEBOOK SE TRABAJA SOLO CON 3 ESTACIONES DE CALIDAD DEL AIRE Y EN NINGUN MOMENTO SE ESCRIBE NI LEE DE LOS ARCHIVOS (aunque el codigo se encuentra disponible en formato Markdown).

> [Configuracion del notebook](#config)
>
> [ESTACIONES DE CALIDAD DEL AIRE](#sitesAQ)
>    
>    * [Estaciones de españa](#espanha)
>
>         1) [Datos de las estaciones de ```saqgetr```](#estacionesSaqgetr)
>
>         2)[Datos geograficos de las estaciones](#estacionGeo)
>
>    * [Estudio de los Datos de las Estaciones](#stdio)
>
>    * [Guardar datos en csv](#saveAQ)
>
> [ESTACIONES DE LA AEMET](#sitesMto)
>
>    * [Guardar datos en csv](#saveMto)
>
> [INFORMACION RELEVANTE](#countdata)
>
>    * [Obtener los datos de Calidad del Aire](#getAQ)
>
>    * [Contar valores de Calidad del Aire](#countAQ)
>
>    * [Contar valores Meteorologicos](#countMto)
>
>    * [Agrupar todos los datos principales](#groupAll)


---
---

### Configuracion del notebook <a id="config"></a>

Cargamos todos los paquetes necesarios para ejecutar el notebook

In [ ]:
# Loading
suppressMessages(library(saqgetr))
suppressMessages(library(tidyverse))
suppressMessages(library(lubridate))
suppressMessages(library(worldmet))
suppressMessages(library(openxlsx))

## Parametros de los datos <a id="param"></a>

In [ ]:
# contaminantes a estudiar
pollutants <- c("no", "no2", "o3", "pm10", "pm2.5")

# fechas de inicio y final de toma de datos
start_dt <- ymd_hms("2010-01-01 00:00:00")
end_dt <- ymd_hms("2020-12-31 00:00:00")

# Archivos de los datos
# NO SE TRABAJA CON ARCHIVOS
sitesAQ.fl = NULL# "../data/csv/sitesAQ.csv"                                           
dataAQ.fl = NULL# "../data/csv/dataAQ.csv"

sitesMto.fl = NULL# "../data/csv/sitesMto.csv"                                         
dataMto.fl = NULL# "../data/csv/dataMto.csv"                                          
years = 2010:2020

final.fl = NULL# "../data/csv/nn_sites.csv"

# Guardar los datos de la calidad del aire
save.data = FALSE

## ESTACIONES DE CALIDAD DEL AIRE <a id="sitesAQ"></a>

### Estaciones de España <a id="espanha"></a>

#### Datos geograficos de las estaciones <a id="estacionGeo"></a>

Seleccionamos aquellas estaciones que se encuentren en un núcleo con una
población mayor o igual a cien mil habitantes `poblacion >= 100 000 hab`.

En la hoja 8 `ciudades-100000-A-JA` de la base de datos de poblaciones (`estaciones-CA.xlsx`) aparecen todas las estaciones de trafico de las ciudades de mas de 100000 habitantes con sus nombres.

In [ ]:
file <- "../data/xlsx/estaciones-CA-JA.xlsx"
sheets <- c("todas", "traffic", "traffic-urban", "traffic-urban-2020",
            "traffic-suburban", "traffic-suburban-2020",
            "ciudades-100000", "ciudades-100000-A")
# xlsx files
sites.100mil <- read.xlsx(file, sheet=sheets[8])

In [ ]:
head(sites.100mil)

#### Datos de las estaciones de `saqgetr` <a id="estacionesSaqgetr"></a>

Importamos la informacion de las estaciones de calidad de aire de españa obtenidas de la base de datos y filtramos segun los criterios de estudio.

| Variable | Valores |
|-------|-----------|
|Contaminantes| $NO$, $NO_2$, $O_3$, $PM_{10}$|
|Fecha Inicio| 01 Enero 2015 |
|Fecha Final | 31 Diciembre 2020 |
| Site Type | traffic |
|site area | urban |

In [ ]:
# obtener datos de CA de España. Salen los códigos de las estaciones
# de Calidad de aire (941)
spain.sites <- get_saq_sites() %>%
    filter(country == "spain",
           site %in% sites.100mil$"Código.estación",
           site_type == "traffic",
           site_area == "urban",
           date_start <= start_dt,
           date_end >= end_dt,
           ) %>%
    select(site, site_name, latitude, longitude, elevation, 
           country, site_type, site_area, date_start, date_end)

In [ ]:
# SOLO SE TRABAJA CON 3 ESTACIONES DE CALIDAD DEL AIRE
spain.sites <- spain.sites[1:3, ]

In [ ]:
removed.sites <- (nrow(sites.100mil)-1) - nrow(spain.sites)

print(paste("Se han eliminado", removed.sites, "estaciones"))
print(paste("Quedan", nrow(spain.sites), "estaciones para el estudio"))

Agrupamos en un solo data.frame toda la informacion relevante de las estaciones de estudio

In [ ]:
sites.info <- get_saq_processes() %>%
    filter(site %in% spain.sites$site,
           variable %in% pollutants,
           date_start <= start_dt,
           #date_end >= end_dt,
          ) %>%
    select(process, site, variable, variable_long, 
           period, unit, observation_count
          )

sites.geo <- sites.100mil %>%
    select("Municipio", "Población", 
           "Estación.tráfico", "Código.estación") %>%
    rename(site = "Código.estación",)

sitesAQ <- merge(x = sites.info, y = sites.geo, by = "site", all.x = TRUE)


In [ ]:
# Convert charactes into factors
sitesAQ <- mutate_if(sitesAQ, is.character, as.factor)

In [ ]:
head(sitesAQ)

### Guardar Datos en csv <a id="saveAQ"></a>

```R
write.csv(sitesAQ, sitesAQ.fl, row.names=FALSE)
```

---
---

## ESTACIONES DE LA AEMET <a id="sitesMto"></a>

Se pueden localizar las estaciones meteo mas cercanas a una estacion de CA mediante mapa

Obtenemos la informacion de las estaciones de calidad del aire del estudio

```R
sitesAQ <- read.csv(sitesAQ.fl)
head(sitesAQ)
```

Obtenemos la informacion de las 3 estaciones meteorologicas mas cercanas a cada estacion de calidad del aire

In [ ]:
sitesMto <- data.frame()
sites.lv <- levels(sitesAQ$site)

for (i in 1:length(sites.lv)) {
    print(sites.lv[i])
    mto <- getMeta(lat = sitesAQ[sitesAQ$site == sites.lv[i], ]$latitude[1], 
                   lon = sitesAQ[sitesAQ$site == sites.lv[i], ]$longitude[1], 
                   end.year = "current",
                   n = 3, returnMap = F)
    mto$siteAQ <- sites.lv[i]
    sitesMto <- rbind(sitesMto, mto)
}

In [ ]:
sitesMto <- mutate_if(sitesMto, is.character, as.factor)

In [ ]:
head(sitesMto)

### Guardar Datos en csv <a id="saveMto"></a>

```R
write.csv(sitesMto, sitesMto.fl, row.names=FALSE)
```

---
---

## INFORMACION RELEVANTE <a id="countdata"></a>

Calcular el numero de datos disponibles para cada estacion y almacenar en una tabla el codigo de cada estacion junto con dicho valor

### Obtener los datos de Calidad del Aire <a id="getAQ"></a>

```R
if (file.exists(sitesAQ.fl)) {
    sitesAQ <- read.csv(sitesAQ.fl, stringsAsFactors = TRUE)
}
```

```R
if (file.exists(dataAQ.fl)) {
    dataAQ <- read.csv(dataAQ.fl, stringsAsFactors = TRUE)
} else {
```

In [ ]:
dataAQ <- get_saq_observations(site = levels(sitesAQ$site),
                             variable = pollutants,
                             valid_only = TRUE,
                             start = start_dt,
                             #end = end_dt,
                             verbose = TRUE
                            )
dataAQ <- mutate_if(dataAQ, is.character, as.factor)

# Save data in a file for each station 
if (save.data) {
    print("Saving Data...")
    write.csv(dataAQ, dataAQ.fl, row.names=FALSE)

    split.by.site(dataAQ, site.lv="all",
                  folder="../data/csv/dataAQ/")
}
#}

In [ ]:
head(dataAQ)

### Contar valores de Calidad del Aire <a id="countAQ"></a>

In [ ]:
numCount <- c()
for (st in levels(sitesAQ$site)) {
    numCount <- c(numCount, nrow(dataAQ[dataAQ$site == st, ]))
}

nn.sitesAQ <- data.frame(siteAQ = levels(sitesAQ$site),
                         countAQ = numCount)

Se anhaden los municipios a los que pertenece cada estacion dentro de la informacion relevante

In [ ]:
municipios <- data.frame()

for (i in 1:nrow(nn.sitesAQ)) {
    municipios[i, "Municipio"] <- sitesAQ[nn.sitesAQ[i, "siteAQ"] == sitesAQ$site,][, "Municipio"][1]
}
nn.sitesAQ <- cbind(municipios, nn.sitesAQ)

In [ ]:
rm(dataAQ, numCount)

nn.sitesAQ <- mutate_if(nn.sitesAQ, is.character, as.factor)

head(nn.sitesAQ)

### Obtener los datos Meteorologicos <a id="getMto"></a>

```R
if (file.exists(sitesMto.fl)) {
    sitesMto <- read.csv(sitesMto.fl, stringsAsFactors=TRUE)
}
```

``` R
if (file.exists(dataMto.fl)) {
    dataMto <- read.csv(dataMto.fl, stringsAsFactors=TRUE)
} else {
```

In [ ]:
# Get Data from worlmet
dataMto <- importNOAA(code = levels(sitesMto$code),
                      year = years,
                      hourly = FALSE,
                      n.cores = 4,
                      quiet = FALSE,
                      path = NA
                     )
dataMto <- mutate_if(dataMto, is.character, as.factor)

# Save data in a file for each station 
if (save.data) {
    print("Saving Data...")
    write.csv(dataMto, dataMto.fl, row.names=FALSE)

    split.by.site(dataMto, site.lv="all",
                  folder="../data/csv/dataMto/")
}
#}

### Contar valores Meteorologicos <a id="countMto"></a>

In [ ]:
countMto <- NA                                                                                          
codeMto <- NA
code.lv <- levels(dataMto$code)

for (i in 1:length(code.lv)) {
    codeMto <- rbind(codeMto, code.lv[i])
    
    countMto <- rbind(countMto, colSums(!is.na(
                                        dataMto[dataMto$code == code.lv[i],
                                        !colnames(dataMto) %in% c("date",
                                                                  "station",
                                                                  "latitude",
                                                                  "longitude",
                                                                  "elev"
                                                                 )])
                                        )
                                / nrow(dataMto[dataMto$code == code.lv[i], ])
                     )
    countMto[i+1, "code"] <- nrow(dataMto[dataMto$code == code.lv[i], ])
}
countMto <- data.frame(countMto[complete.cases(countMto), ]) %>%
                rename(countMto = code)

codeMto <- data.frame(codeMto[complete.cases(codeMto), ])
colnames(codeMto) <- c("siteMto")

countMto <- cbind(codeMto, countMto)

nn.sitesMto <- merge(x = sitesMto %>%
                        rename(siteMto=code) %>%
                        select(siteMto, dist, siteAQ),
                     y = countMto,
                     by = "siteMto", all=TRUE
                    )

rm(dataMto, countMto, sitesMto, codeMto)

In [ ]:
nn.sitesMto <- mutate_if(nn.sitesMto, is.character, as.factor)
head(nn.sitesMto)

---
---

## TABLA CON RELACION NN

In [ ]:
nn.sites <- merge(x = nn.sitesAQ,
                  y = nn.sitesMto,
                  by = "siteAQ", all = TRUE)
nn.sites <- mutate_if(nn.sites, is.character, as.factor)
head(nn.sites)

```R
write.csv(nn.sites, final.fl, row.names=FALSE)
```